In [ ]:
import pandas as pd

#importar e fazer um pré-tratamento de dados, como remover valores nulos e colunas de índices aleatórios.
df = pd.read_csv('clientes.csv')
display(df.info())
display(df)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# codificar as colunas que contem texto. 
for column in df.columns:
    if df[column].dtype == 'object' and column != 'score_credito':
        df[column] = LabelEncoder().fit_transform(df[column])

# verificando se as colunas foram codificadas.
for column in df.columns:
    if df[column].dtype == 'object' and column != 'score_credito':
        print(column)

display(df)

In [ ]:
# separar colunas em x e y 
# x = dados que serão analisados pela inteligência artificial.
x = df.drop(columns=['score_credito','id_cliente'])

# y = as respostas das análises.
y = df['score_credito']

display(x)
display(y)

In [ ]:
# separar as amostras em treino e teste.
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state=1)

In [ ]:
# criar os modelos de inteligência artificial.
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

model_decision_tree = RandomForestClassifier() # modelo de árvore de decisão.
model_knn = KNeighborsClassifier() # modelo nearest neibors - vizinhos próximos.

# treinando os modelos.
model_decision_tree.fit(x_train, y_train)
model_knn.fit(x_train, y_train)


In [ ]:
# se o modelo chutar tudo, qual será a acurácia do modelo?
count_scores = df['score_credito'].value_counts()

for line in count_scores.to_frame().index:
    print(f'se chutar tudo {line}: {((count_scores[line] / sum(count_scores)) * 100):.2f}%')

# Verificar a acurácia das inteligências artificiais.
prevision_dicision_tree = model_decision_tree.predict(x_test)
prevision_knn = model_knn.predict(x_test)

from sklearn.metrics import accuracy_score

print(f'acuracia da arvore de decisao: {(accuracy_score(y_test, prevision_dicision_tree) * 100):.2f}%')
print(f'acuracia do knn: {(accuracy_score(y_test, prevision_knn) * 100):.2f}%')

In [ ]:
# quais as características mais importantes para definir o score de crédito.
columns = list(x_test.columns)
importance = pd.DataFrame(index=columns, data=model_decision_tree.feature_importances_, columns=['Importância'])
importance = importance * 100
print('Quais as caracteristicas mais importantes segundo arvore de decisao')
display(importance.sort_values(by='Importância', ascending=False))

In [ ]:
df_new_clients = pd.read_csv('novos_clientes.csv')
display(df_new_clients)

# codificar as colunas que contem texto. 
for column in df_new_clients.columns:
    if df_new_clients[column].dtype == 'object' and column != 'score_credito':
        df_new_clients[column] = LabelEncoder().fit_transform(df_new_clients[column])

# verificando se as colunas foram codificadas.
for column in df_new_clients.columns:
    if df_new_clients[column].dtype == 'object' and column != 'score_credito':
        print(column)

In [ ]:
# fazendo as 2 IA`s compararem e julgarem o score de crédito dos clientes.
predict_dicision_tree = model_decision_tree.predict(df_new_clients)
predict_knn = model_knn.predict(df_new_clients)

print('segundo a arvore de decisao')
df_new_clients['score_credito'] = predict_dicision_tree
display(df_new_clients)
print('segundo o knn')
df_new_clients['score_credito'] = predict_knn
display(df_new_clients)